<a href="https://colab.research.google.com/github/FrancescoTorella/progettoLabAI/blob/main/training1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 53.3 MB/s eta 0:00:00


In [ ]:
import torch
import torchvision
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms.functional as TF
import os
import rasterio
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import DataLoader
import shutil
import matplotlib.pyplot as plt
import time
import random
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r drive/MyDrive/progettoLabAi3/train/mask/val_set /content/mask

In [ ]:
class MultispectralDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir   #directory delle immagini di train senza label
        self.mask_dir = mask_dir    #directory delle label
        self.transform = transform  #trasformazione
        self.images = os.listdir(image_dir) #lista di stringhe contenente tutti i nomi delle immagini senza label

    def __len__(self):
        return len(self.images)    #lunghezza della stringa che contiene tutti i path, quindi grandezza del dataset di train

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index]) #ricostruisce il persorso completo dell'immagine corrispondente all'indice fornito
        #mask_path = os.path.join(self.mask_dir, self.images[index].replace(".tif", "_mask.tiff").replace("PS-RGBNIR","Buildings")) #ricostruisce il percorso completo della label corrispondente all'indice fornito, inoltre sostituisce .jpg con .gif, qui si assume che il nome della label sia uguale a quello dell'immagine con l'estensione eccetto, devo modificare questa riga in seguito
        string = self.images[index]
        mask_name = string.replace('PS-RGBNIR','Buildings').replace(".tif","_mask.tiff")
        #print("STRINGA"+ string)
        #tile_index= indice_numero = max(string.rfind('_'), string.rfind(' '))
        #mask_name = string[:tile_index + 1] + "mask_" + string[tile_index + 1:]
        #print("FINALE: "+ mask_name)
        #mask_name = mask_name.replace("11 mask_2.tiff","mask_11.tiff")
        #mask_name = mask_name.replace("12 mask_2.tiff","mask_12.tiff")
        #mask_name = mask_name.replace("13 mask_2.tiff","mask_23.tiff")
        #mask_name = mask_name.replace("21 mask_2.tiff","mask_21.tiff")
        #mask_name = mask_name.replace("22 mask_2.tiff","mask_22.tiff")
        #mask_name = mask_name.replace("23 mask_2.tiff","mask_23.tiff")
        #mask_name = mask_name.replace("31 mask_2.tiff","mask_31.tiff")
        #mask_name = mask_name.replace("32 mask_2.tiff","mask_32.tiff")
        #mask_name = mask_name.replace("33 mask_2.tiff","mask_33.tiff")
        mask_path = os.path.join(self.mask_dir, mask_name) #ricostruisce il percorso completo della label corrispondente all'indice fornito, inoltre sostituisce .jpg con .gif, qui si assume che il nome della label sia uguale a quello dell'immagine con l'estensione eccetto, devo modificare questa riga in seguito
        with rasterio.open(img_path) as src:
            image = src.read().transpose((1, 2, 0)).astype(np.float32) #trasforma l'immagine da formato (C, H, W) a (H, W, C)
        #with rasterio.open(mask_path) as src:
            #mask = src.read().transpose((1,2,0)).astype(np.float32)
        with rasterio.open(mask_path) as src:
          mask = src.read().astype(np.float32)
        #mask = np.array(Image.open(mask_path).convert("L"), dtype=np.long)


        #print("ciao")
        #plt.imshow(mask)
        #plt.title(f"Mask 1:")
        #plt.axis('off')
        #plt.show()
        #apre l'immagine e la converte in scala di grigi, inoltre la trasforma in un array numpy di float32
        mask[mask == 255.0] = 1.0 #sostituisce tutti i valori 255 con 1, serve per la normalizzazione
        mask = mask.squeeze(0)
        #plt.imshow(mask)
        #plt.title(f"Mask 2:")
        #plt.axis('off')
        #plt.show()

        if self.transform is not None: #se è stata definita una trasformazione
            augmentations = self.transform(image=image, mask=mask) #applica le trasformazioni all'immagine e alla label, restituisce un dizionario con chiavi image e mask
            image = augmentations["image"] #estrai dal dizionario l'immagine e la assegna alla variabile image
            mask = augmentations["mask"] #estrai dal dizionario la label e la assegna alla variabile mask

        #plt.imshow(mask)
        #plt.title(f"Mask 3:")
        #plt.axis('off')
        #plt.show()

        return image, mask

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

In [ ]:
#salva il checkpoint del modello in un file .pth.tar, state è un dizionario che contiene lo stato del modello e dell'ottimizzatore, filename è il nome del file in cui salvare il checkpoint
def save_checkpoint(state, filename="/content/drive/My Drive/progettoLabAi3/computations/saved/my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

#carica il checkpoint del modello da un file .pth.tar, checkpoint è un dizionario che contiene lo stato del modello e dell'ottimizzatore, model è l'istanza del modello, checkpoint è un dizionario che contiene lo stato del modello e dell'ottimizzatore salvato in un file .pth.tar, model è l'istanza del modello in cui caricare il checkpoint
def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):

    #show_random_image_from_directory(train_maskdir)

    #crea un'istanza della classe MultispectralDataset, che rappresenta il dataset di addestramento, image_dir è la directory delle immagini di addestramento, mask_dir è la directory delle label di addestramento, transform è la trasformazione da applicare al dataset
    train_ds = MultispectralDataset(
        image_dir=train_dir,
        mask_dir=train_maskdir,
        transform=train_transform,
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers, #numero di processi in background da utilizzare per il caricamento dei dati
        pin_memory=pin_memory, #se True, i dati verranno copiati in memoria GPU prima di essere passati al modello, nel mio caso è inutile settarlo a True perché non ho una GPU dedicata
        shuffle=True, #se True, i dati verranno mescolati prima di essere passati al modello in ogni epoca di addestramento
    )

    #crea un'istanza della classe MultispectralDataset, che rappresenta il dataset di validazione, image_dir è la directory delle immagini di validazione, mask_dir è la directory delle label di validazione, transform è la trasformazione da applicare al dataset
    val_ds = MultispectralDataset(
        image_dir=val_dir,
        mask_dir=val_maskdir,
        transform=val_transform,
    )

    #stessa cosa di train_loader ma per il dataset di validazione, infatti passiamo val_ds come primo argomento, però shuffle=False perché non serve mescolare i dati di validazione
    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader

def check_accuracy(loader, model, loss_fn, file, limited=False,limit=32,device="cuda"):
    model.eval()
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    precision = 0
    recall = 0
    f1 = 0
    val_loss = 0
    iteration = 1
    print(f'interations: {len(loader)}')
    with torch.no_grad():
        for x, y in loader:
            print(f'Iteration number: {iteration}')

            iteration += 1

            if limited:
              if(iteration > limit):
                break



            x = x.to(device)
            y = y.to(device).unsqueeze(1)

            out = model(x)
            val_loss += loss_fn(out, y).item()
            preds = torch.sigmoid(out)
            preds = (preds > 0.5)
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / ((preds + y).sum() + 1e-8) #preds or y
            precision += precision_score(y.cpu().numpy().flatten(), preds.cpu().numpy().flatten(), zero_division=1)
            recall += recall_score(y.cpu().numpy().flatten(), preds.cpu().numpy().flatten(), zero_division=1)
            f1 += f1_score(y.cpu().numpy().flatten(), preds.cpu().numpy().flatten(), zero_division=1)





    if limited:
      denominator = min(limit, len(loader))
    else:
      denominator = len(loader)

    val_loss /= denominator
    accuracy = num_correct / num_pixels * 100
    dice = dice_score / denominator
    precision_avg = precision / denominator
    recall_avg = recall / denominator
    f1_avg = f1 / denominator

    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Dice score: {dice}")
    print(f"Precision: {precision_avg}")
    print(f"Recall: {recall_avg}")
    print(f"F1 Score: {f1_avg}")

    # Salva i risultati su un file di testo
    with open(file, "a") as f:
        f.write(f"Validation Loss: {val_loss:.4f}\n")
        f.write(f"Accuracy: {accuracy:.2f}\n")
        f.write(f"Dice score: {dice}\n")
        f.write(f"Precision: {precision_avg}\n")
        f.write(f"Recall: {recall_avg}\n")
        f.write(f"F1 Score: {f1_avg}\n")
        f.write("\n")  # Riga vuota per separare le epoche

    model.train()


def save_predictions_as_imgs(loader, model, folder="/content/drive/My Drive/progettoLabAi3/computations/saved/images/",
                             device="cuda"):
    model.eval()  # Imposta il modello in modalità di valutazione
    print(f"Saving images to {folder}")
    iteration = 1
    for idx, (x, y) in enumerate(loader):
        print("Iteration: ", iteration)
        iteration += 1
        x = x.to(device=device)
        y = y.to(device=device).unsqueeze(1)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()

        # Concatenare le immagini (maschera originale e predizione) lungo l'asse verticale
        combined = torch.cat((y, preds), dim=2)

        # Salvare l'immagine combinata
        torchvision.utils.save_image(combined, f"{folder}/comparison_{idx}.png")

    model.train()  # Ripristina il modello in modalità di addestramento

In [ ]:
def visualize_image_mask(image, mask, index):
    # Utilizza solo i primi tre canali (R, G, B)
    image_rgb = image[:, :, :3]

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(image_rgb)
    axs[0].set_title(f"Image {index}")
    axs[1].imshow(mask, cmap='gray')
    axs[1].set_title(f"Mask {index}")
    plt.show()

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def write_description(description, file):
  with open(file, "a") as f:
    f.write(f"Descrizione: {description}\n\n\n\n")

In [ ]:
# Hyperparameters etc.
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 20
NUM_WORKERS = 4
IMAGE_HEIGHT = 300
IMAGE_WIDTH = 300
STEP_SIZE = 18
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = "/content/images/train_set"
TRAIN_MASK_DIR = "/content/mask/train_set"
VAL_IMG_DIR = "/content/images/val_set"
VAL_MASK_DIR = "/content/mask/val_set"
MODEL_DIRECTORY = "/content/drive/My Drive/progettoLabAi3/computations/saved/model15/"
MODEL_DESCRIPTION = "modello identico al primo ma con la media e varianza calcolate da me"


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def train_fn(loader, model, optimizer, loss_fn, scaler,file):
    loop = tqdm(loader)  # Aggiunge una barra di avanzamento



    # Enumerate restituisce una tupla contenente un contatore e il valore restituito dall'iterabile
    for batch_idx, (data, targets) in enumerate(loop):

        #print(f'data.shape: {data.shape}, data.dtype: {data.dtype}')
        #print(f'targets.shape: {targets.shape}, targets.dtype: {targets.dtype}')
        data = data.to(device=DEVICE)  # Trasferisce i dati sulla GPU
        targets = targets.float().to(device=DEVICE)  # Trasferisce le label sulla GPU e aggiunge una dimensione

        with torch.cuda.amp.autocast():
            predictions = model(data).squeeze(1) #predictions è un tensore di predizioni, ad esempio (16, 1, 300, 300)
            loss = loss_fn(predictions, targets)



        # backward
        optimizer.zero_grad() #azzera i gradienti
        scaler.scale(loss).backward() #calcola i gradienti e scala i gradienti
        scaler.step(optimizer) #aggiorna i pesi
        scaler.update() #descala i pesi


        loop.set_postfix(loss=loss.item())
        last_loss = loss.item()  # Memorizza l'ultima loss calcolata

    # Salva l'ultima loss calcolata su un file di testo
    with open(file, "a") as f:
        f.write(f"Train Loss: {last_loss:.4f}\n")



def main():

    set_seed(42)

    print("Creating transforms")
    train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            #A.CenterCrop(height=896,width=896),
            #A.RandomCrop(height=320,width=320),
            #A.Rotate(limit=90, p=0.5),
            #A.HorizontalFlip(p=0.5),
            #A.VerticalFlip(p=0.5),
            A.Normalize(
                #mean=[84.882515, 114.9725, 122.400505, 264.03476],
                #std=[112.35377, 132.05618, 154.0384, 294.54465],
                mean=[81.2788, 111.04898, 117.07683, 261.56107],
                std=[110.35951, 129.69858 ,151.0430,296.55923],
                #max_pixel_value=65535.0,
            ),
            ToTensorV2(),
        ],
    )
    val_transforms = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            #A.CenterCrop(height=896,width=896),
            #A.RandomCrop(height=320,width=320),
            A.Normalize(
                #mean=[81.27881, 111.04898, 117.07683, 261.56107],
                #std=[110.35951, 129.69858, 151.043, 296.55923],
                mean=[84.882515, 114.9725, 122.400505, 264.03476],
                std=[112.35377, 132.05618, 154.0384, 294.54465],
                #max_pixel_value=65535.0,
            ),
            ToTensorV2(),
        ],
    )

    write_description(MODEL_DESCRIPTION, file= MODEL_DIRECTORY +"outputs/outputs.txt")

    print("Creating model")
    model = UNET(in_channels=4, out_channels=1).to(DEVICE)
    print("Model created")
    print("creating loss function and optimizer")
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE)
    print("loss function and optimizer created")

    print("Creating data loaders")
    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )


    print("data loaders created")
    if LOAD_MODEL:
        load_checkpoint(torch.load(MODEL_DIRECTORY +"my_checkpoint.pth.tar"), model)

    print("checking accuracy on the validation set before training")
    #check_accuracy(
     #       val_loader,
     #       model,
     #       loss_fn,
     #       file= MODEL_DIRECTORY +"outputs/outputs.txt",
     #       limited = True,
     #       device=DEVICE
    #)
    #save_predictions_as_imgs(
     #         val_loader, model, folder=MODEL_DIRECTORY +"images/", device=DEVICE
    #)
    scaler = torch.cuda.amp.GradScaler()

    print("Starting training")
    for epoch in range(NUM_EPOCHS):
        print(f"epoch {epoch}")
        print("starting training")
        train_fn(train_loader, model, optimizer, loss_fn, scaler, file= MODEL_DIRECTORY +"outputs/outputs.txt")
        print("training done")

        print("saving model")
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint,filename = MODEL_DIRECTORY + "my_checkpoint.pth.tar")
        print("model saved")

        print("checking accuracy on the validation set after training")
        check_accuracy(
            val_loader,
            model,
            loss_fn,
            file= MODEL_DIRECTORY +"outputs/outputs.txt",
            limited = False,
            device=DEVICE
        )

        if((epoch) % 2 == 0):
          print("saving predictions to folder")
          save_predictions_as_imgs(
              val_loader, model, folder=MODEL_DIRECTORY +"images/", device=DEVICE
          )

        scheduler.step()

if __name__ == "__main__":
    main()

Creating transforms
Creating model


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Model created
creating loss function and optimizer
loss function and optimizer created
Creating data loaders
data loaders created
checking accuracy on the validation set before training
Starting training
epoch 0
starting training


  0%|          | 0/149 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|██████████| 149/149 [02:10<00:00,  1.14it/s, loss=0.302]


training done
saving model
=> Saving checkpoint
model saved
checking accuracy on the validation set after training
interations: 32
Iteration number: 1
Iteration number: 2
Iteration number: 3
Iteration number: 4
Iteration number: 5
Iteration number: 6
Iteration number: 7
Iteration number: 8
Iteration number: 9
Iteration number: 10
Iteration number: 11
Iteration number: 12
Iteration number: 13
Iteration number: 14
Iteration number: 15
Iteration number: 16
Iteration number: 17
Iteration number: 18
Iteration number: 19
Iteration number: 20
Iteration number: 21
Iteration number: 22
Iteration number: 23
Iteration number: 24
Iteration number: 25
Iteration number: 26
Iteration number: 27
Iteration number: 28
Iteration number: 29
Iteration number: 30
Iteration number: 31
Iteration number: 32


...

Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
epoch 1
starting training


100%|██████████| 149/149 [02:02<00:00,  1.21it/s, loss=0.196]


training done
saving model
=> Saving checkpoint
model saved
checking accuracy on the validation set after training
interations: 32
Iteration number: 1
Iteration number: 2
Iteration number: 3
Iteration number: 4
Iteration number: 5
Iteration number: 6
Iteration number: 7
Iteration number: 8
Iteration number: 9
Iteration number: 10
Iteration number: 11
Iteration number: 12
Iteration number: 13
Iteration number: 14
Iteration number: 15
Iteration number: 16
Iteration number: 17
Iteration number: 18
Iteration number: 19
Iteration number: 20
Iteration number: 21
Iteration number: 22
Iteration number: 23
Iteration number: 24
Iteration number: 25
Iteration number: 26
Iteration number: 27
Iteration number: 28
Iteration number: 29
Iteration number: 30
Iteration number: 31
Iteration number: 32
Validation Loss: 0.2661
Accuracy: 92.74
Dice score: 0.0
Precision: 1.0
Recall: 0.0
F1 Score: 0.0
epoch 2
starting training


  0%|          | 0/149 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  8%|▊         | 12/149 [00:12<02:21,  1.03s/it, loss=0.202]


KeyboardInterrupt: 